In [1]:
!uv add cfbd

Resolved 102 packages in 2.01s
 Downloaded pydantic
Prepared 3 packages in 1.54s
Installed 3 packages in 296ms
 + aenum==3.1.16
 + cfbd==5.13.2
 + pydantic==1.10.26


In [ ]:
import os
from cfbd.configuration import Configuration
from dotenv import load_dotenv

load_dotenv()
configuration = Configuration(
    host = "https://api.collegefootballdata.com",
    access_token = os.environ["CFBD_API_KEY"]
)

In [ ]:
from cfbd.api_client import ApiClient
from cfbd.api.games_api import GamesApi
from pprint import pprint


with ApiClient(configuration) as api_client:
    api_instance = GamesApi(api_client)
    year = 2025
    week = None
    season_type = None
    classification = None
    team = None
    home = None
    away = None
    conference = None
    id = None

    try:
        api_response = api_instance.get_games(year=year, week=week, season_type=season_type, classification=classification, team=team, home=home, away=away, conference=conference, id=id)
        print("The response of GamesApi->get_games:\n")
        pprint(api_response)
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401756846, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2025, 8, 23, 16, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=True, attendance=None, venue_id=3504, venue='Aviva Stadium', home_id=2306, home_team='Kansas State', home_conference='Big 12', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=21, home_line_scores=[0, 7, 0, 14], home_postgame_win_probability=0.20154441893100739, home_pregame_elo=1669, home_postgame_elo=1666, away_id=66, away_team='Iowa State', away_conference='Big 12', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=24, away_line_scores=[7, 0, 7, 10], away_postgame_win_probability=0.7984555810689926, away_pregame_elo=1616, away_postgame_elo=1619, excitement_index=6.029094160739601, highlights='', notes='Aer Lingus College Football Classic'),
 Game(id=401767476, season=2025,

In [16]:
print(api_response[0].home_team)
print(api_response[0].away_team)
print(api_response[0].home_points)
print(api_response[0].away_points)

Kansas State
Iowa State
21
24


In [80]:
import pandas as pd
from enum import Enum


df = pd.DataFrame([g.to_dict() for g in api_response])

# Convert enums to strings
df["seasonType"] = df["seasonType"].apply(
    lambda x: x.value if isinstance(x, Enum) else x
).astype("string")
df["awayClassification"] = df["awayClassification"].apply(
    lambda x: x.value if isinstance(x, Enum) else x
).astype("string")
df["homeClassification"] = df["homeClassification"].apply(
    lambda x: x.value if isinstance(x, Enum) else x
).astype("string")

# Drop columns
df = df.drop(columns=[
    "startTimeTBD", "completed", "attendance", "highlights", "venue", "awayPostgameWinProbability", "awayPregameElo",
    "awayPostgameElo", "excitementIndex", "notes", "homePostgameWinProbability", "homePregameElo", "homePostgameElo"
])

# Modify types
df["venueId"] = df["venueId"].astype("Int64")
df["awayPoints"] = df["awayPoints"].astype("Int64")
df["homePoints"] = df["homePoints"].astype("Int64")

In [81]:
df.head()

,id,season,week,seasonType,startDate,neutralSite,conferenceGame,venueId,homeId,homeTeam,homeConference,homeClassification,homePoints,homeLineScores,awayId,awayTeam,awayConference,awayClassification,awayPoints,awayLineScores
0,401756846,2025,1,regular,2025-08-23 16:00:00+00:00,True,True,3504,2306,Kansas State,Big 12,fbs,21,"[0, 7, 0, 14]",66,Iowa State,Big 12,fbs,24,"[7, 0, 7, 10]"
1,401767476,2025,1,regular,2025-08-23 17:00:00+00:00,False,True,3779,2447,Nicholls,Southland,fcs,20,"[10, 3, 0, 7]",2916,Incarnate Word,Southland,fcs,6,"[3, 0, 3, 0]"
2,401760371,2025,1,regular,2025-08-23 20:00:00+00:00,False,False,6501,2439,UNLV,Mountain West,fbs,38,"[0, 14, 10, 14]",304,Idaho State,Big Sky,fcs,31,"[10, 7, 7, 7]"
3,401767126,2025,1,regular,2025-08-23 20:30:00+00:00,False,False,4765,2502,Portland State,Big Sky,fcs,0,"[0, 0, 0, 0]",2627,Tarleton State,UAC,fcs,42,"[7, 7, 14, 14]"
4,401756847,2025,1,regular,2025-08-23 22:30:00+00:00,False,False,3833,2305,Kansas,Big 12,fbs,31,"[14, 10, 7, 0]",278,Fresno State,Mountain West,fbs,7,"[7, 0, 0, 0]"


In [146]:
df["homeConference"].unique()

array(['Big 12', 'Southland', 'Mountain West', 'Big Sky',
       'Conference USA', 'SWAC', 'Lone Star', 'American Athletic',
       'Big Ten', 'GLIAC', None, 'MVFC', 'Mid-American',
       'Pennsylvania State Athletic', 'Gulf South', 'Sun Belt', 'Pioneer',
       'Carolinas', 'Northern Sun', 'Great Midwest Athletic', 'ACC',
       'Mountain East', 'MEAC', 'Big South-OVC', 'SEC', 'Great Lakes',
       'Southern', 'Rocky Mountain', 'Mid America', 'CAA', 'Patriot',
       'South Atlantic', 'SIAC', 'CIAA', 'Northeast 10',
       'FBS Independents', 'FCS Independents', 'UAC', 'Pac-12',
       'USA South', 'Mid Atlantic', 'Liberty League', 'Great American',
       'Ohio', 'Landmark Conference', "Presidents'",
       'Southern Collegiate Athletic', 'Wisconsin', 'American Southwest',
       'Midwest', 'Old Dominion', 'NEWMAC', 'Commonwealth Coast',
       'Empire 8', 'MSCAC', 'Centennial', 'NACC', 'North Coast', 'NEC',
       'Heartland', 'New Jersey', 'Southern Athletic', 'CCIW',
       'Amer

In [ ]:
def get_conference_colley_matrix(conf : str) -> pd.DataFrame:
    conf_df = df[(df["homeConference"] == conf) & (df["awayConference"] == conf) & (df["week"] < 15) & (df["seasonType"] == "regular") & (df["conferenceGame"])]
    conf_df = conf_df.drop(columns=["homeConference", "awayConference", "conferenceGame", "homeClassification", "awayClassification", "seasonType", "season"])

    conf_teams = sorted(conf_df["homeTeam"].unique())
    n = len(conf_teams)
    number_of_conference_games = int(len(conf_df) * 2 / n)
    wins = [0] * n

    C = np.zeros((n, n), dtype=int)
    for _, row in conf_df.iterrows():
        i = conf_teams.index(row["homeTeam"])
        j = conf_teams.index(row["awayTeam"])
        if row["homePoints"] > row["awayPoints"]:
            wins[i] += 1
        else:
            wins[j] += 1

        C[i, j] -= 1
        C[j, i] -= 1

    np.fill_diagonal(C, number_of_conference_games + 2)
    b = np.array([1 + (2*i - number_of_conference_games) for i in wins]).reshape((n, 1))
    r = np.linalg.solve(C, b)

    conf_colley_df = pd.DataFrame({
        "Team": conf_teams,
        "Wins": wins,
        "Losses": [number_of_conference_games - i for i in wins],
        "Colley Rating": r.flatten()
    })
    return conf_colley_df.sort_values(by="Colley Rating", ascending=False)

In [187]:
get_conference_colley_matrix("SEC")

,Team,Wins,Losses,Colley Rating
0,Alabama,7,1,1.124368
4,Georgia,7,1,1.071183
10,Ole Miss,7,1,0.939980
9,Oklahoma,6,2,0.932006
13,Texas,6,2,0.867994
14,Texas A&M,7,1,0.857950
15,Vanderbilt,6,2,0.852769
12,Tennessee,4,4,0.494635
8,Missouri,4,4,0.449551
6,LSU,3,5,0.363778


In [188]:
get_conference_colley_matrix("Big 12")

,Team,Wins,Losses,Colley Rating
2,BYU,8,1,1.156969
12,Texas Tech,8,1,1.107735
14,Utah,7,2,0.976623
1,Arizona State,6,3,0.822057
0,Arizona,6,3,0.720632
6,Houston,6,3,0.669313
11,TCU,5,4,0.606062
7,Iowa State,5,4,0.570406
4,Cincinnati,5,4,0.562375
9,Kansas State,5,4,0.493709


In [189]:
get_conference_colley_matrix("Big Ten")

,Team,Wins,Losses,Colley Rating
9,Ohio State,9,0,1.201636
10,Oregon,8,1,1.185861
1,Indiana,9,0,1.173163
15,USC,7,2,0.924532
4,Michigan,7,2,0.842580
2,Iowa,6,3,0.791703
0,Illinois,5,4,0.569144
6,Minnesota,5,4,0.548620
16,Washington,5,4,0.547052
8,Northwestern,4,5,0.437397


In [190]:
get_conference_colley_matrix("ACC")

,Team,Wins,Losses,Colley Rating
14,Virginia,7,1,1.031583
3,Duke,6,2,0.897340
7,Miami,6,2,0.838115
5,Georgia Tech,6,2,0.833464
10,Pittsburgh,6,2,0.814923
11,SMU,6,2,0.807165
16,Wake Forest,4,4,0.561343
6,Louisville,4,4,0.552756
8,NC State,4,4,0.544460
1,California,4,4,0.485453


In [192]:
get_conference_colley_matrix("Mid-American")

,Team,Wins,Losses,Colley Rating
12,Western Michigan,7,1,1.103429
8,Miami (OH),6,2,0.971074
11,Toledo,6,2,0.923024
10,Ohio,6,2,0.826175
4,Central Michigan,5,3,0.646541
0,Akron,4,4,0.457642
3,Buffalo,4,4,0.443886
1,Ball State,3,5,0.420254
6,Kent State,4,4,0.384868
5,Eastern Michigan,3,5,0.365779


In [195]:
get_conference_colley_matrix("American Athletic")

,Team,Wins,Losses,Colley Rating
5,Navy,7,1,1.035366
6,North Texas,7,1,1.034884
10,Tulane,7,1,0.982593
8,South Florida,6,2,0.856568
2,East Carolina,6,2,0.800775
13,UTSA,4,4,0.545123
4,Memphis,4,4,0.524807
0,Army,4,4,0.464634
9,Temple,3,5,0.364634
3,Florida Atlantic,3,5,0.342989


In [196]:
get_conference_colley_matrix("Sun Belt")

,Team,Wins,Losses,Colley Rating
5,James Madison,8,0,1.236597
8,Old Dominion,6,2,0.847863
12,Troy,6,2,0.829915
6,Louisiana,5,3,0.733100
1,Arkansas State,5,3,0.653846
10,Southern Miss,5,3,0.653846
2,Coastal Carolina,5,3,0.614530
3,Georgia Southern,4,4,0.527972
11,Texas State,3,5,0.369464
7,Marshall,3,5,0.327506


In [197]:
get_conference_colley_matrix("Mountain West")

,Team,Wins,Losses,Colley Rating
1,Boise State,6,2,0.926840
6,New Mexico,6,2,0.879253
9,UNLV,6,2,0.855759
7,San Diego State,6,2,0.844349
4,Hawai'i,5,3,0.637084
3,Fresno State,5,3,0.632877
10,Utah State,4,4,0.553056
0,Air Force,3,5,0.303345
5,Nevada,2,6,0.185981
8,San José State,2,6,0.109420


In [199]:
get_conference_colley_matrix("Conference USA")

,Team,Wins,Losses,Colley Rating
3,Kennesaw State,7,1,1.068654
2,Jacksonville State,7,1,1.055510
11,Western Kentucky,6,2,0.876308
1,Florida International,5,3,0.761312
7,Missouri State,5,3,0.688727
5,Louisiana Tech,5,3,0.635891
0,Delaware,4,4,0.470714
4,Liberty,3,5,0.367049
6,Middle Tennessee,2,6,0.188377
8,New Mexico State,2,6,0.064769


In [200]:
get_conference_colley_matrix("MVFC")

,Team,Wins,Losses,Colley Rating
4,North Dakota State,8,0,1.283333e+00
6,South Dakota,6,2,8.500000e-01
0,Illinois State,5,3,6.666667e-01
3,North Dakota,5,3,6.666667e-01
9,Youngstown State,5,3,6.500000e-01
7,South Dakota State,4,4,5.000000e-01
8,Southern Illinois,4,4,5.000000e-01
5,Northern Iowa,1,7,1.155866e-16
1,Indiana State,1,7,1.105363e-16
2,Murray State,1,7,-1.166667e-01
